# Mid-term Project

In [76]:
# Importing necessary libraries
import os
import pandas as pd
import numpy as np
from pymongo import MongoClient
import json
import datetime
import mysql.connector
from sqlalchemy import create_engine
import sqlalchemy
import pymongo

In [78]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.30
Running PyMongo Version: 4.10.1


In [80]:
#declare & assign connection variables 
mysql_args = {
    "uid" : "root",
    "pwd" : "unicornTree98*!",
    "hostname" : "localhost",
    "dbname" : "sakila"
}

mongodb_args = {
    "user_name" : "paigelee",
    "password" : "Password",
    "cluster_name" : "Cluster1",
    "cluster_subnet" : "axtgz",
    "cluster_location" : "atlas", # "local"
    "db_name" : "sakila"
}

In [29]:
import mysql.connector

conn = mysql.connector.connect(
    host= host_name,  
    user= user_id,  
    password= pwd)
cursor = conn.cursor()

# Create new database
cursor.execute("CREATE DATABASE IF NOT EXISTS adventureworks_dw")
cursor.execute("USE adventureworks_dw")
print("Data warehouse database created.")

Data warehouse database created.


In [53]:
cursor.execute("USE adventureworks_dw;")

# Call the stored procedure to populate the dim_date table
cursor.execute("CALL PopulateDateDimension('2000-01-01', '2010-12-31');")
conn.commit()  # Commit changes to ensure data is inserted

print("dim_date table populated successfully.")

InternalError: Unread result found

In [55]:
cursor.execute("USE adventureworks;")

# Query the customer table
query = "SELECT * FROM customer;"
cursor.execute(query)

InternalError: Unread result found

In [59]:
def connect_mysql():
    return mysql.connector.connect(
        host=mysql_credentials["host"],
        user=mysql_credentials["user"],
        password=mysql_credentials["password"],
        database=mysql_credentials["database"]
    )

def load_to_mysql(df, table_name):
    engine = create_engine(f"mysql+mysqlconnector://{mysql_credentials['user']}:{mysql_credentials['password']}@{mysql_credentials['host']}/{mysql_credentials['database']}")
    df.to_sql(table_name, con=engine, if_exists="replace", index=False)

def load_to_mongodb(json_file, db_name, collection_name):
    client = MongoClient("your_mongodb_connection_string")  # Update with MongoDB connection string
    db = client[db_name]
    collection = db[collection_name]
    with open(json_file) as file:
        data = json.load(file)
    collection.insert_many(data)
    print("Data loaded to MongoDB")

In [ ]:
# Load JSON file into MongoDB
load_to_mongodb(json_path, "adventureworks", "customer_collection")  # Replace with your MongoDB database/collection

In [ ]:
# Load CSV data into a DataFrame
product_df = pd.read_csv(csv_path)

# Load the DataFrame to MySQL as a new dimension table
load_to_mysql(product_df, "dim_product")
print("Product dimension table created in MySQL")

In [ ]:
# Connect to MongoDB and read data from a collection
client = MongoClient("your_mongodb_connection_string")  # Replace with your MongoDB connection string
db = client["adventureworks"]
collection = db["customer_collection"]
data = list(collection.find())
customer_df = pd.DataFrame(data)

# Drop MongoDB’s default _id column
customer_df = customer_df.drop(columns=["_id"])

# Write MongoDB data to MySQL as a dimension table
load_to_mysql(customer_df, "dim_customer")
print("Customer data loaded from MongoDB to MySQL as dim_customer.")

In [ ]:
# Read transaction data from MySQL (assuming you have an `orders` table)
conn = connect_mysql()
transaction_query = "SELECT * FROM orders"  # Update with actual transaction table name
transaction_df = pd.read_sql(transaction_query, conn)

# Lookup operations to replace business codes with surrogate keys
transaction_df = transaction_df.merge(customer_df[['customer_id', 'customer_key']], on="customer_id", how="left")
transaction_df = transaction_df.merge(product_df[['product_id', 'product_key']], on="product_id", how="left")

# Write to MySQL as a fact table
load_to_mysql(transaction_df, "fact_sales")
print("Fact table created as fact_sales.")

In [ ]:
-- Summarize sales amount by customer
SELECT c.customer_name, SUM(f.sales_amount) AS total_sales
FROM fact_sales f
JOIN dim_customer c ON f.customer_id = c.customer_id
GROUP BY c.customer_name;

-- Summarize units sold by product
SELECT p.product_name, SUM(f.units_sold) AS total_units
FROM fact_sales f
JOIN dim_product p ON f.product_id = p.product_id
GROUP BY p.product_name;

-- Group sales by month
SELECT d.calendar_year_month, SUM(f.sales_amount) AS total_sales
FROM fact_sales f
JOIN dim_date d ON f.date_key = d.date_key
GROUP BY d.calendar_year_month;